In [4]:
# NOTE: requires the *.zip archives in 1_data/3_eo_rapid/ to be unzipped

In [5]:
from hydromt_sfincs import SfincsModel
import hydromt
from os.path import join, basename
import xarray as xr
import numpy as np
import glob
import os
import sys

In [6]:
# requires the model to be setup in step 2_experiment/21_setup_sfincs_models.ipynb
root = r'../../3_models/SFINCS/00_base_100m'
mod0 = SfincsModel(root, mode='r')

In [ ]:
# read and resample images to model domain
ddir = r'../../1_data/3_eo_rapid'
dates = ['20190319', '20190320', '20210125']
types = ['flooding', 'Dry']

rm_fns = []
for date in dates:
    for name in types:
        fns = glob.glob(join(ddir, date, f'{name}*.tif'))
        da_lst = []
        for fn in fns:
            print(fn)
            da_obs0 = hydromt.open_raster(fn).load().astype(np.int8)
            da_obs0.raster.set_nodata(-1)
            try:
                da_obs0 = da_obs0.raster.reproject_like(mod0.staticmaps, method='max')
                da_lst.append(da_obs0)
            except IndexError:
                da_obs0.close()
                rm_fns.append(fn)
                print('out of bounds')
                pass
            
        if len(da_lst) > 0:
            print(f'concatenate {len(da_lst)} files')
            da = xr.concat(da_lst, dim='img').max('img').load().astype(np.uint8)
            da = da.where(da==1,0)
            da.raster.set_nodata(0)
            da.raster.to_raster(join(root, 'gis', f'{name.lower()}_{date}.tif'), compress='deflate')

if len(rm_fns) > 0:
    print(f'deleting {len(rm_fns)} files')
    for fn in rm_fns:
        os.unlink(fn)

In [7]:
# read and resample processed sentinel 1 images to model domain
ddir = r'../../1_data/3_eo_sentinel1'
events = {
    'idai': ['20190319', '20190320'], 
    'eloise':['20210125','20210126']
}
fns = glob.glob(join(ddir, f'sofala*.tif'))
for event, dates in events.items():
    da_obs_lst = []
    for date in dates:
        name = f'sofala_floodmap_RefinedLee_{date}_10m.tif'
        fn = join(ddir, name)
        da_obs0 = hydromt.open_raster(fn).load().astype(np.int8)
        da_obs0.raster.set_nodata(-1)
        da_obs0 = da_obs0.raster.reproject_like(mod0.staticmaps, method='max')
        # da_obs0 = da_obs0.where(da_obs0==1,0)
        # da_obs0.raster.set_nodata(-1)
        da_obs_lst.append(da_obs0)
        da_obs0.raster.to_raster(join(root, 'gis', f'sofala_floodmap_{date}.tif'), compress='deflate')
    da_obs_max = xr.concat(da_obs_lst, dim='time').max('time')
    da_obs0.raster.to_raster(join(root, 'gis', f'sofala_floodmap_{event}_max.tif'), compress='deflate')

    

nodata value missing for ../../1_data/3_eo_sentinel1\sofala_floodmap_RefinedLee_20190319_10m.tif
nodata value missing for ../../1_data/3_eo_sentinel1\sofala_floodmap_RefinedLee_20190320_10m.tif
nodata value missing for ../../1_data/3_eo_sentinel1\sofala_floodmap_RefinedLee_20210125_10m.tif
nodata value missing for ../../1_data/3_eo_sentinel1\sofala_floodmap_RefinedLee_20210126_10m.tif
